In [96]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [97]:
# set up spark
conf = SparkConf()\
    .setAppName("data migration")\
    .setMaster("local")\
    .set("spark.driver.extraClassPath", "../../jar/*")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [98]:
# extract
def extract():

    try:

        uid = '{YOUR USER ID ON POSTGRESQL}'
        pwd = '{YOUR PSSWORD}'
        host = 'localhost'
        port = '5432' #this is the default port
        db = '{YOUR DB NAME}'
        driver = "org.postgresql.Driver"
        url = f"jdbc:postgresql://{host}:5432/{db}?user={uid}&password={pwd}"

        tables_name = ['category_db', 'education_db', 'marital_db', 'status_db', 'customer_data_history']
        dfs = []

        for table in tables_name:

            query = f"SELECT * FROM {table}"

            df = spark.read.\
                format("jdbc").\
                options(driver=driver, user=uid, password = pwd, url=url, query =query).\
                load()
            
            dfs.append(df)


    except Exception as e:

        print("error to get database : "+str(e))

    finally:

        print("successfull to get database")
        return (dfs, tables_name)
        

In [99]:
#transform
def transform(extract):

    dfs = extract[0]
    tables_name = extract[1]

    idx = tables_name.index('customer_data_history')
    dfs[idx] = dfs[idx].filter(dfs[idx].Customer_Age < 50)

    return (dfs, tables_name)

In [100]:
#load
def load(transform):

    dfs = transform[0]
    tables_name = transform[1]

    try:

        uid = '{YOUR USER ID ON SQL SERVER}'
        pwd = '{YOUR PASSWORD}'
        host = '{DEVICE NAME}\SQLEXPRESS'
        port = '1433' #this is the default port
        db = 'rakamin'
        driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
        url = f"jdbc:sqlserver://{host}:{port};databaseName={db}; user={uid}; password={pwd};encrypt=true;trustServerCertificate=true;"

        for df, table in zip(dfs, tables_name):

            df.write.mode("overwrite")\
            .format("jdbc")\
            .options(driver=driver, user=uid, password=pwd, url=url, dbtable=table)\
            .save()

    except Exception as e:

        print("failed to load tables into database "+str(e))

    finally:

        print("successfull to load tables into database")

In [101]:
# run
extract = extract()
transform = transform(extract=extract)
load(transform=transform)

successfull to get database
successfull to load tables into database
